In [16]:
from _Setup import *
from data_functions import *

import tensorflow as tf
from keras import layers

## Import train and test data

In [17]:
train_data = pd.read_csv(rawTrainFile)
test_data = pd.read_csv(rawTestFile)

C:\Users\Trevo\AppData\Local\Temp\ipykernel_63964\3702163432.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(rawTestFile)


# Train data preparation

### Remove Columns w/ > 95% initial missing values and B/F imputation

In [18]:
# Identify columns with more than 90% missing values

missing_percentage = train_data.isnull().mean() * 100

cols_to_drop_for_missing_percentage = missing_percentage[missing_percentage > 95].index

# Drop the identified columns

train_truncated = train_data.drop(columns=cols_to_drop_for_missing_percentage)
test_truncated = test_data.drop(columns = cols_to_drop_for_missing_percentage)
cols_to_drop_for_missing_percentage

Index(['EtCO2', 'HCO3', 'SaO2', 'AST', 'Alkalinephos', 'Chloride',
       'Bilirubin_direct', 'Lactate', 'Phosphate', 'Bilirubin_total',
       'TroponinI', 'PTT', 'Fibrinogen'],
      dtype='object')

In [19]:
#forward imputation
train_forward = forward_fill_by_patient(train_truncated)
test_forward = forward_fill_by_patient(test_truncated)
train_forward.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Hgb,WBC,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,0.0,...,12.0,14.2,NaN,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,0.0,...,12.0,14.2,NaN,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,-2.5,...,12.0,14.2,NaN,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,-2.5,...,12.0,14.2,NaN,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [20]:
#backward imputation
pre_imputed_train = backward_fill_by_patient(train_forward)
pre_imputed_test = backward_fill_by_patient(test_forward)
pre_imputed_train.head()  

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Hgb,WBC,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,78.0,99.5,35.25,108.5,84.5,69.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0.0,1.0,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0.0,1.0,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0.0,1.0,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,-2.5,...,12.0,14.2,162.0,57.03,1.0,0.0,1.0,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,-2.5,...,12.0,14.2,162.0,57.03,1.0,0.0,1.0,-4.25,5.0,0.0


In [21]:
# Identify columns with more than 90% missing values

missing_percentage = pre_imputed_train.isnull().mean() * 100

cols_to_drop_for_missing_percentage2 = missing_percentage[missing_percentage > 80].index

cols_to_drop_for_missing_percentage2

Index([], dtype='object')

## MissForest imputation

In [22]:
naPercentages = pd.DataFrame(pre_imputed_train.isna().sum() / pre_imputed_train.shape[0], columns=['Percentage'])
naPercentages.sort_values(by = ['Percentage'], ascending=False)           

,Percentage
BaseExcess,0.637390
FiO2,0.501017
PaCO2,0.485635
pH,0.471871
Unit1,0.391502
Unit2,0.391502
DBP,0.166844
Calcium,0.096244
Magnesium,0.087443
WBC,0.041200


In [24]:
# Solve a bug within the test data
pre_imputed_test['HR'] = pd.to_numeric(pre_imputed_test['HR'], errors='coerce')

In [25]:
imputer = MissForest()

imputer_input = pre_imputed_train.drop('SepsisLabel', axis = 1)
imputed_data = imputer.fit_transform(imputer_input)
imputed_test = imputer.transform(pre_imputed_test)

imputed_data['SepsisLabel'] = pre_imputed_train['SepsisLabel']

imputed_data.isna().sum()

100%|██████████| 5/5 [02:30<00:00, 30.10s/it]


patient        0
time           0
HR             0
O2Sat          0
Temp           0
SBP            0
MAP            0
DBP            0
Resp           0
BaseExcess     0
FiO2           0
pH             0
PaCO2          0
BUN            0
Calcium        0
Creatinine     0
Glucose        0
Magnesium      0
Potassium      0
Hct            0
Hgb            0
WBC            0
Platelets      0
Age            0
Gender         0
Unit1          0
Unit2          0
HospAdmTime    0
ICULOS         0
SepsisLabel    0
dtype: int64

## Fixing uni1 and unit2 problem

In [26]:
# Convert to binary values
imputed_data['Unit1'] = imputed_data['Unit1'].apply(lambda x: 1 if x >= 0.5 else 0)
imputed_data['Unit2'] = imputed_data['Unit2'].apply(lambda x: 1 if x >= 0.5 else 0)

imputed_test['Unit1'] = imputed_test['Unit1'].apply(lambda x: 1 if x >= 0.5 else 0)
imputed_test['Unit2'] = imputed_test['Unit2'].apply(lambda x: 1 if x >= 0.5 else 0)

# Print the unique values again to verify
print("Unique values in Unit1 after transformation:")
print(imputed_data['Unit1'].unique())

print("\nUnique values in Unit2 after transformation:")
print(imputed_data['Unit2'].unique())


Unique values in Unit1 after transformation:
[0 1]

Unique values in Unit2 after transformation:
[1 0]


# Workflow:

1. Train/Validation Split
2. Minority oversampling by patient

--- Separate Files Here ---

3. Build RNN
4. Evaluate Performance



In [27]:
# Create variable to see if patient ever has sepsis
imputed_data['SepsisPatient'] = imputed_data.groupby('patient')['SepsisLabel'].transform(lambda x: 1 if (x == 1).any() else 0)


# Create create [patient, SepsisEver] pair to take stratified sample
patientsList = imputed_data.drop_duplicates(subset=['patient', 'SepsisPatient'])[['patient', 'SepsisPatient']]

X_sample = patientsList['patient']
y_sample =  patientsList['SepsisPatient']

# Take stratified sample based upon whether individual ever has Sepsis
patient_labels_train, patient_labels_validate, _, _ = train_test_split(X_sample, y_sample, test_size=0.2, stratify=y_sample)


In [28]:
# Reexpand Train Data
train_split_data = imputed_data[imputed_data['patient'].isin(patient_labels_train)].drop('SepsisPatient', axis = 1)
validate_split_data = imputed_data[imputed_data['patient'].isin(patient_labels_validate)].drop('SepsisPatient', axis = 1)
validate_split_data.head()

,patient,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Hgb,WBC,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,16426,1,78.0,99.5,35.25,108.5,84.5,69.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0,1,-4.25,1.0,0.0
1,16426,2,78.0,99.5,35.25,108.5,84.5,69.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0,1,-4.25,2.0,0.0
2,16426,3,80.0,99.0,36.20,113.0,87.0,73.0,12.0,0.0,...,12.0,14.2,162.0,57.03,1.0,0,1,-4.25,3.0,0.0
3,16426,4,79.0,100.0,36.50,112.0,83.0,68.0,12.0,-2.5,...,12.0,14.2,162.0,57.03,1.0,0,1,-4.25,4.0,0.0
4,16426,5,73.0,100.0,36.50,115.0,80.0,64.0,11.0,-2.5,...,12.0,14.2,162.0,57.03,1.0,0,1,-4.25,5.0,0.0


### TODO : Minority oversampling?

### TODO : Feature Engineering

In [30]:
featured_train = FeatureEngineering(train_split_data)
featured_validate = FeatureEngineering(validate_split_data)
featured_test = FeatureEngineering(imputed_test)

In [ ]:
# Scale train data 

exclude_scaling_cols = ['time', 'patient', 'Gender', 'Unit1', 'Unit2', 'Fever', 'TCP', 'TCA', 'LEU']

# Separate columns into those to scale and those to exclude
cols_to_scale = featured_train.drop(columns=exclude_scaling_cols + ['SepsisLabel']).columns
scaled_data = featured_train[cols_to_scale]

# Scale the selected columns
scaler = StandardScaler()
scaled_scaled_data = pd.DataFrame(scaler.fit_transform(scaled_data), columns=cols_to_scale)

# Combine scaled data with excluded columns and the target variable
scaled_train = pd.concat(
    [scaled_scaled_data, 
     featured_train[exclude_scaling_cols].reset_index(drop=True), 
     featured_train['SepsisLabel'].reset_index(drop=True)], 
    axis=1
)

# Display the resulting DataFrame
print(scaled_train.head())

       time        HR     O2Sat      Temp       SBP       MAP       DBP  \
0 -0.868508  0.894411  0.267736 -0.352793 -1.256814 -1.206428 -1.231939   
1 -0.834664  0.894411  0.267736 -0.352793 -1.256814 -1.206428 -1.231939   
2 -0.800820  0.722508  0.426239 -0.164119 -1.256814 -1.206428 -1.231939   
3 -0.766977  1.467422 -0.366275 -0.040475  0.230541  0.564318 -0.133339   
4 -0.733133  1.238217 -1.317293 -0.194014 -1.019700  0.442198  0.672302   

       Resp  BaseExcess      FiO2  ...     Shock  patient  Gender  Unit1  \
0 -1.096202    0.062501  0.002003  ... -0.001072    18075     0.0      0   
1 -1.096202    0.062501  0.002003  ... -0.001072    18075     0.0      0   
2 -1.193059    0.061499  0.001131  ... -0.001072    18075     0.0      0   
3 -1.193059    0.060669  0.001097  ... -0.001073    18075     0.0      0   
4 -1.193059    0.060669 -0.002932  ... -0.001072    18075     0.0      0   

   Unit2  Fever  TCP  TCA  LEU  SepsisLabel  
0      1      0    0    1    0          0.0  


In [33]:
# Scale Validation Data
scaled_val = featured_validate[cols_to_scale]

scaled_val_data = pd.DataFrame(scaler.transform(scaled_val), columns=cols_to_scale)
scaled_val = pd.concat([scaled_val_data, 
                         featured_validate[exclude_scaling_cols].reset_index(drop = True), 
                         featured_validate['SepsisLabel'].reset_index(drop = True)],
                         axis = 1)

scaled_val.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU,SepsisLabel
0,-0.868508,-0.366211,0.743245,-2.223329,-0.653250,0.106366,0.379342,-1.289916,0.056664,0.056749,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
1,-0.834664,-0.366211,0.743245,-2.223329,-0.653250,0.106366,0.379342,-1.289916,0.056664,0.056749,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
2,-0.800820,-0.251609,0.584742,-0.908420,-0.459247,0.259017,0.672302,-1.289916,0.056664,-0.001415,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
3,-0.766977,-0.308910,0.901748,-0.493186,-0.502359,0.014776,0.306102,-1.289916,-0.831026,-0.001415,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
4,-0.733133,-0.652717,0.901748,-0.493186,-0.373024,-0.168405,0.013141,-1.483631,-0.831026,-0.013048,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0


In [34]:
# Scale test data

scaled_test_cols = featured_test.drop(columns = exclude_scaling_cols).columns
scaled_test = featured_test[scaled_test_cols]

scaled_test_data = pd.DataFrame(scaler.transform(scaled_test), columns=scaled_test_cols)
scaled_test = pd.concat([scaled_test_data, 
                         featured_test[exclude_scaling_cols].reset_index(drop = True)], 
                         axis = 1)

In [71]:
# Save to CSV's 
rnnTrainFile = os.path.join(imputedDataPath, "rnn_train.csv")
rnnValFile = os.path.join(imputedDataPath, "rnn_val.csv")
rnnTestFile = os.path.join(imputedDataPath, "rnn_test.csv")

scaled_train.to_csv(rnnTrainFile, index=False)
scaled_val.to_csv(rnnValFile, index=False)
scaled_test.to_csv(rnnTestFile, index=False)

# IF THE PIPELINE BREAKS START RUN HERE

In [1]:
from _Setup import *
from data_functions import *

import tensorflow as tf
from keras import layers

scaled_train = pd.read_csv(rnnTrainFile)
scaled_val = pd.read_csv(rnnValFile)
scaled_test = pd.read_csv(rnnTestFile)

In [2]:
scaled_train.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU,SepsisLabel
0,-0.868508,0.894411,0.267736,-0.352793,-1.256814,-1.206428,-1.231939,-1.096202,0.062501,0.002003,...,-0.001072,18075,0.0,0,1,0,0,1,0,0.0
1,-0.834664,0.894411,0.267736,-0.352793,-1.256814,-1.206428,-1.231939,-1.096202,0.062501,0.002003,...,-0.001072,18075,0.0,0,1,0,0,1,0,0.0
2,-0.800820,0.722508,0.426239,-0.164119,-1.256814,-1.206428,-1.231939,-1.193059,0.061499,0.001131,...,-0.001072,18075,0.0,0,1,0,0,1,0,0.0
3,-0.766977,1.467422,-0.366275,-0.040475,0.230541,0.564318,-0.133339,-1.193059,0.060669,0.001097,...,-0.001073,18075,0.0,0,1,0,0,1,0,0.0
4,-0.733133,1.238217,-1.317293,-0.194014,-1.019700,0.442198,0.672302,-1.193059,0.060669,-0.002932,...,-0.001072,18075,0.0,0,1,0,0,1,0,0.0


### Window Shifting

In [3]:
scaled_val['patient'].value_counts().min()

8

In [4]:

# Parameters
window_size = 3  # Length of each sliding window
step_size = 1    # Step size to slide the window

# Sliding window function to generate windows (using specific columns)
def sliding_window(df, columns, window_size, step_size):
    windows = []
    for patient_id, group in df.groupby('patient'):
        for i in range(0, len(group) - window_size + 1, step_size):
            windows.append(group[columns].iloc[i:i + window_size].values)
    return np.array(windows)

# TODO : Fix this
features = scaled_train.drop(['patient', 'SepsisLabel', 'Unit2'], axis = 1).columns

windowed_data = sliding_window(scaled_train, features, window_size, step_size)



In [5]:
windowed_data.shape

(823727, 3, 32)

### Create window indices
Create the corresponding data in the same order as the dataframe, groupby function does this by default for keeping everything in order

In [6]:
def sliding_window_indices(df, columns, window_size, step_size):
    windows = []
    for patient_id, group in df.groupby('patient'):
        for i in range(0, len(group) - window_size + 1, step_size):
            windows.append(group[['patient', 'time']].iloc[i:i + window_size].values)
    return np.array(windows)


# NOTE : Only use this for testing purposes, smote will render this irrelevant
#        Function will be needed for the test and validate sets
window_indices = sliding_window_indices(scaled_train, scaled_train.columns, window_size, step_size)

### Creating Sepsis Labels corresponding to the maximum of the window:
Do this to produce results of the window

In [8]:
def create_sepsis_label_tensor(df, window_size, step_size):
    sepsis_labels = []
    
    for patient_id, group in df.groupby('patient'):
       # For each window in the patient, repeat the SepsisLabel for the window length
        for i in range(0, len(group) - window_size + 1, step_size):
            sepsis_labels.append(group['SepsisLabel'].iloc[i:i + window_size].max())
    
    return np.array(sepsis_labels)

sepsis_tensor = create_sepsis_label_tensor(scaled_train, window_size, step_size)

In [9]:
windowed_data.shape

(823727, 3, 32)

In [10]:
sepsis_tensor.shape

(823727,)

### Minority oversampling
Do the minority oversampling here after the windowing to avoid strange regrouping/shifting again

In [11]:
# TODO : Play with the class imbalance parameter

# Reshape window data for input into SMOTE and ease of use with RNN
reshaped_windowed_data = windowed_data.reshape(windowed_data.shape[0],-1)

smote = SMOTE(sampling_strategy = 'auto', random_state = 8)
smote_windowed_data, smote_sepsis_tensor = smote.fit_resample(reshaped_windowed_data, sepsis_tensor)


In [12]:
smote_windowed_data.shape

(1617218, 96)

In [38]:
# Resize input for prep into RNN
reshaped_smote_windowed_data = smote_windowed_data.reshape((smote_windowed_data.shape[0], window_size, len(features)))
reshaped_smote_windowed_data.shape

(1617218, 3, 32)

In [39]:
# Convert to a tensor
data_tensor = tf.convert_to_tensor(reshaped_smote_windowed_data, dtype=tf.float32)

# TODO : Likely remove this line
# Pad sequences to make them uniform length (if necessary)
# padded_data = tf.keras.preprocessing.sequence.pad_sequences(windowed_data, padding="post", dtype='float32')

# Convert SepsisLabel to tensor
output_tensor = tf.convert_to_tensor(smote_sepsis_tensor, dtype=tf.float32)

In [ ]:
# TODO : Tune this model

# Define an RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.0, input_shape=(window_size, len(features))),
    tf.keras.layers.SimpleRNN(50),
    tf.keras.layers.SimpleRNN(40),
    tf.keras.layers.SimpleRNN(30),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile and train the model (using the SepsisLabel as target)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(data_tensor, output_tensor, epochs=5, batch_size=100)  # Using SepsisLabel as the target

print("Training complete.")

Epoch 1/5
16173/16173 ━━━━━━━━━━━━━━━━━━━━ 50s 3ms/step - accuracy: 0.7157 - loss: 0.5577
Epoch 2/5
16173/16173 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.7561 - loss: 0.5063
Epoch 3/5
16173/16173 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.7607 - loss: 0.4984
Epoch 4/5
16173/16173 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step - accuracy: 0.7629 - loss: 0.4932
Epoch 5/5
16173/16173 ━━━━━━━━━━━━━━━━━━━━ 65s 4ms/step - accuracy: 0.7634 - loss: 0.4911
Training complete.


In [42]:
train_predictions = model.predict(data_tensor)

50539/50539 ━━━━━━━━━━━━━━━━━━━━ 121s 2ms/step


### Create pipeline for validation data
Feed validation data through the pipeline

In [46]:
windowed_val = sliding_window(scaled_val, features, window_size, step_size)
window_val_indices = sliding_window_indices(scaled_val, scaled_val.columns, window_size, step_size)
sepsis_val_tensor = create_sepsis_label_tensor(scaled_val, window_size, step_size)

In [47]:
val_predictions = model.predict(windowed_val)

6500/6500 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step


In [48]:
val_predictions.shape

(208000, 1)

In [49]:
window_val_indices

array([[[ 1.00000000e+00, -8.68507614e-01],
        [ 1.00000000e+00, -8.34663984e-01],
        [ 1.00000000e+00, -8.00820355e-01]],

       [[ 1.00000000e+00, -8.34663984e-01],
        [ 1.00000000e+00, -8.00820355e-01],
        [ 1.00000000e+00, -7.66976725e-01]],

       [[ 1.00000000e+00, -8.00820355e-01],
        [ 1.00000000e+00, -7.66976725e-01],
        [ 1.00000000e+00, -7.33133096e-01]],

       ...,

       [[ 2.82290000e+04, -2.93165913e-01],
        [ 2.82290000e+04, -2.59322283e-01],
        [ 2.82290000e+04, -2.25478654e-01]],

       [[ 2.82290000e+04, -2.59322283e-01],
        [ 2.82290000e+04, -2.25478654e-01],
        [ 2.82290000e+04, -1.91635024e-01]],

       [[ 2.82290000e+04, -2.25478654e-01],
        [ 2.82290000e+04, -1.91635024e-01],
        [ 2.82290000e+04, -1.57791395e-01]]])

In [112]:
import numpy as np
import pandas as pd

# nx3x2 matrix (n samples, each with a 3x2 matrix)
larger_matrix = np.round(np.random.rand(n, 3, 2), 2)

# nx1 matrix (n corresponding prediction values for each sample)
predictions_matrix = np.round(np.random.rand(n, 1),2)

def dewindow(larger_matrix, predictions_matrix, threshold = 0.85):

    n = larger_matrix.shape[0]

    array_data = []

    # Map each entry in the larger matrix to the corresponding prediction in the smaller matrix
    for i in range(n):
        # Extract the 3x2 matrix for the current sample
        sample_matrix = larger_matrix[i]

        # Extract the corresponding prediction value from the smaller matrix
        prediction_value = predictions_matrix[i, 0]  # This is a scalar

        # Create a new list of arrays for each column in the dataframe
        column1 = []

        for row in sample_matrix:
            # Each row is a 2-element array (e.g., [value1, value2])
            # Append the array with the corresponding prediction value
            column1.append(np.append(row, prediction_value))  # Append prediction_value to the row

        # Add the columns as arrays to the array_data
        array_data.append((column1))

    # Create DataFrame with each column as an array
    # Convert the list of arrays into a pandas DataFrame
    df = pd.DataFrame(array_data)


    melted_df = df.melt(value_name="Values")

    melted_df_cols = ["patient", "time", "SepsisLabel"]

    df_expanded = melted_df['Values'].apply(pd.Series)

    df_expanded.columns = melted_df_cols
    idx_max_sepsis = df_expanded.groupby(['patient', 'time'])['SepsisLabel'].idxmax()

    # Filter the dataframe to keep only these rows
    df_max_sepsis = df_expanded.loc[idx_max_sepsis]
    df_max_sepsis['SepsisLabel'] = (df_max_sepsis['SepsisLabel'] > threshold).astype(int)

    return df_max_sepsis

test = dewindow(larger_matrix, predictions_matrix)
test

,patient,time,SepsisLabel
2,0.16,0.27,1
5,0.19,0.74,0
11,0.20,0.75,0
8,0.27,0.75,0
10,0.29,0.24,0
6,0.32,0.73,0
3,0.34,0.48,0
13,0.46,0.14,0
1,0.48,0.98,0
7,0.49,0.91,1


In [113]:
dewindowed_val = dewindow(window_val_indices, val_predictions)

In [114]:
scaled_val.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU,SepsisLabel
0,-0.868508,-0.366211,0.743245,-2.223329,-0.653250,0.106366,0.379342,-1.289916,0.056664,0.056749,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
1,-0.834664,-0.366211,0.743245,-2.223329,-0.653250,0.106366,0.379342,-1.289916,0.056664,0.056749,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
2,-0.800820,-0.251609,0.584742,-0.908420,-0.459247,0.259017,0.672302,-1.289916,0.056664,-0.001415,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
3,-0.766977,-0.308910,0.901748,-0.493186,-0.502359,0.014776,0.306102,-1.289916,-0.831026,-0.001415,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0
4,-0.733133,-0.652717,0.901748,-0.493186,-0.373024,-0.168405,0.013141,-1.483631,-0.831026,-0.013048,...,-0.001073,16426,1.0,0,1,1,0,0,1,0.0


In [115]:
dewindowed_val.head()

,patient,time,SepsisLabel
0,1.0,-0.868508,0
1,1.0,-0.834664,0
208001,1.0,-0.800820,0
416001,1.0,-0.766977,0
416002,1.0,-0.733133,0


In [116]:
# TODO : move this package up
from sklearn.metrics import f1_score
f1 = f1_score(scaled_val['SepsisLabel'], dewindowed_val['SepsisLabel'])
f1

0.020092135205639117